In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(threshold=np.nan)
from sklearn.preprocessing import scale

exec(open("/Users/raghuramkowdeed/Documents/afp/get_signal_names.py").read())


In [2]:
rps_file = "/Users/raghuramkowdeed/Desktop/rps_signals.csv"
signal_data = pd.read_csv(rps_file)
sig_names = [ x.lower() for x in signal_data.columns ]
signal_data.columns = sig_names

signal_data = signal_data.set_index("date")
#signal_data.dropna(how = "any", axis = 1 ,inplace = True)
signal_data.head()

,permno,gvkey,fyear,sic2,spi,mve_f,bm,ep,cashpr,dy,...,baspread,std_dolvol,std_turn,ill,zerotrade,beta,betasq,rsq1,pricedelay,idiovol
date,,,,,,,,,,,,,,,,,,,,,
19800131,10006,1010,1978,37,0.0000,269.308500,1.180962,0.153022,-32.218678,0.067347,...,0.014234,0.881844,0.635898,2.565667e-08,1.115306e-07,1.060404,1.124456,0.343403,0.029861,0.025576
19800131,10057,1098,1978,36,0.0000,97.372000,0.956692,0.135131,-4.408581,0.036982,...,0.022470,1.368363,2.546787,2.719812e-07,6.199128e-08,1.525993,2.328654,0.307902,0.092675,0.037473
19800131,10103,1012,1978,33,NaN,1.697500,3.362003,0.338144,-17.143817,0.000000,...,0.511667,NaN,NaN,NaN,NaN,1.759501,3.095842,0.096756,0.221852,0.087020
19800131,10137,1279,1978,49,NaN,537.524500,1.330341,0.153238,-87.819837,0.137560,...,0.020899,0.553246,0.740017,1.765620e-08,9.726790e-08,0.492885,0.242936,0.189697,0.125795,0.017540
19800131,10145,1300,1978,99,-0.0031,805.633282,1.579284,0.149248,-22.050470,0.070247,...,0.016947,0.427617,0.657563,2.898901e-09,6.190654e-08,1.139150,1.297663,0.279436,0.024234,0.031201


In [3]:
#fama_french = pd.read_csv("/home/ywu/Documents/workplace/AFP_data/F-F_Research_Data_5_Factors_2x3_daily.CSV", skiprows = 3)
#fama_french.set_index("DATE", inplace = True)
#fama_french.head()

In [4]:
#fama_french_cleaned = fama_french.loc[signal_data.index]
#fama_french_cleaned = fama_french_cleaned[~fama_french_cleaned .index.duplicated(keep='first')]
#fama_french_cleaned.head()

In [5]:
def divide_data(dataset):
    gb = dataset.groupby(dataset.index)
    for x in gb.groups:
        gb.get_group(x).to_csv(str(x)+".csv")

In [6]:
def clean_data(dataset, out_dir):
    dates = np.unique(dataset.index)
    
    for dt in dates:
        print(dt)
        temp = dataset.loc[dt,:]
        print(temp.shape)
        permno = temp["permno"]
        gvkey = temp["gvkey"]
        this_ret = temp['ret'] 
        temp = temp.loc[:,get_signal_names()]
        temp = temp.rank()*1.0
        scaled =  ( temp - temp.mean(axis=0) )/ temp.std(axis=0)
        scaled = scaled.fillna(0)
        scaled = (scaled.rank()*1.0)/scaled.shape[0]
        result = pd.DataFrame(scaled, index = temp.index, columns = temp.columns)
        #dataset = normalize(dataset)
        result["permno"] = permno
        result["gvkey"] = gvkey
        result['ret'] = this_ret
        result.to_csv(out_dir+str(dt)+".csv")

In [7]:
def get_ticker(dataset, num = 1000, col = "mve"):
    return dataset.nlargest(num, col)

In [8]:
#data1 = pd.read_csv("/home/ywu/Documents/Github/afp/data_cleaning/Data/19800131.csv")
#data1.head()

In [9]:
clean_data(signal_data, out_dir='/Users/raghuramkowdeed/Desktop/rps_data_dir/')

19800131
(3482, 149)
19800229
(3482, 149)
19800331
(3519, 149)
19800430
(3512, 149)
19800530
(3529, 149)
19800630
(3550, 149)
19800731
(3576, 149)
19800829
(3623, 149)
19800930
(3755, 149)
19801031
(3735, 149)
19801128
(3731, 149)
19801231
(3741, 149)
19810130
(3741, 149)
19810227
(3730, 149)
19810331
(3725, 149)
19810430
(3711, 149)
19810529
(3710, 149)
19810630
(3713, 149)
19810731
(3707, 149)
19810831
(3726, 149)
19810930
(3809, 149)
19811030
(3790, 149)
19811130
(3804, 149)
19811231
(3821, 149)
19820129
(3815, 149)
19820226
(3838, 149)
19820331
(3861, 149)
19820430
(3858, 149)
19820528
(3873, 149)
19820630
(3888, 149)
19820730
(3884, 149)
19820831
(3930, 149)
19820930
(4069, 149)
19821029
(4062, 149)
19821130
(4067, 149)
19821231
(4087, 149)
19830131
(4087, 149)
19830228
(4075, 149)
19830331
(4108, 149)
19830429
(4100, 149)
19830531
(4093, 149)
19830630
(4093, 149)
19830729
(4095, 149)
19830831
(4100, 149)
19830930
(4171, 149)
19831031
(4164, 149)
19831130
(4167, 149)
19831230
(417

In [10]:
#!mkdir /Users/raghuramkowdeed/Desktop/rps_data_dir/